Dataset with target value in json

Plaats je code voor deze delen in de stappen hieronder. Vergeet ook zeker niet de bijhorende vragen te beantwoorden.

**Opgepast:** Laat de output staan zodat het eenvoudiger is om je bekomen resultaten te interpreteren

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

In [ ]:
class BrainTumorDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        """
        Args:
            img_dir (str): Directory with all the images.
            transform (callable, optional): Optional transformations to apply to the images.
        """
        self.img_dir = img_dir
        self.transform = transform
        self.image_files = os.listdir(img_dir)  # List all image files in the directory

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Get image path
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        # Extract label from filename
        if "tumor" in img_name.lower():
            label = 1
        elif "no_tumor" in img_name.lower():
            label = 0
        else:
            raise ValueError(f"Cannot infer label from filename: {img_name}")
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
# Path to images
img_dir = 'brain_tumor_dataset/images'

# Data transformations for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the custom dataset
full_dataset = BrainTumorDataset(img_dir=img_dir, transform=data_transforms['train'])

# Split into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Set the correct transform for each dataset
train_dataset.dataset.transform = data_transforms['train']  # Apply train transform to train split
val_dataset.dataset.transform = data_transforms['val']      # Apply val transform to validation split

# Create DataLoaders for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Check the number of batches
print(f"Train loader batches: {len(train_loader)}")
print(f"Validation loader batches: {len(val_loader)}")